## <span style="color:#ff5f27">🧬 Train Retrieval Model </span>


## <span style="color:#ff5f27">📝 Imports </span>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import StringLookup, Normalization

import tensorflow_recommenders as tfrs

import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [2]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/11383
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🔪 Feature Selection </span>


In [3]:
users_fg = fs.get_feature_group(
    name="users",
    version=1,
)

videos_fg = fs.get_feature_group(
    name="videos",
    version=1,
)

interactions_fg = fs.get_feature_group(
    name="interactions",
    version=1,
)

---

In [24]:

obs = pd.DataFrame(
    [[generic.person.identifier(mask='####-##-####'), 'like', 'WA546S', '3EL60S', 206]],
    columns=['interaction_id', 'interaction_type', 'user_id', 'video_id', 'watch_time']
)
obs

,interaction_id,interaction_type,user_id,video_id,watch_time
0,9035-15-2410,like,WA546S,3EL60S,206


In [25]:
import numpy as np

interactions_fg.insert(obs)

2024-04-19 15:29:44,878 INFO: 	7 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works/p/11383/fs/11331/fg/12303


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/11383/jobs/named/interactions_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f23cd356110>,
 {
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "interactions_data_suite",
     "run_id": {
       "run_time": "2024-04-19T15:29:44.878362+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "a63410e2-fe61-11ee-a9f8-964546f6b095"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20240419T152944.878315Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.15.12"
     }
   },
   "results": [
     {
       "result": {
         "element_count": 1,
         "missing_count": 0,
         "missing_percent": 0.0,
         "unexpected_count": 0,
         "unexpected_percent": 0.0,
         "unexpected_percent_total": 0.0,
         "unexpected_percent_nonmissing": 0.0,
         "partial_unexpected_list": []
       },
       "exception_info": {
         "raised_exception": false,
         "e

---

In [4]:
QUERY_FEATURES = ["user_id", "gender", "age", "country"]
CANDIDATE_FEATURES = ["video_id", "category", "views", "likes", "video_length"]

In [5]:
# Select features for training data
selected_features = interactions_fg.select(["interaction_id"])\
    .join(users_fg.select(QUERY_FEATURES), on="user_id")\
    .join(videos_fg.select(CANDIDATE_FEATURES), on="video_id")

# Uncomment this if you would like to view your selected features
selected_features.show(5)

Finished: Reading data from Hopsworks, using ArrowFlight (39.67s) 


,interaction_id,user_id,gender,age,country,video_id,category,views,likes,video_length
0,2877-12-0444,RG876M,Female,76,South Africa,6ZY07Y,News,192408,91932,113
1,4380-37-7971,YU710V,Female,59,Gambia,5ZY75O,Comedy,7275,5936,171
2,3714-23-7897,IQ911P,Male,35,Sudan,6NE41R,Dance,66149,45789,238
3,8380-73-5772,BB023A,Male,32,Cyprus,2OL92N,Technology,26029,2383,113
4,7047-00-6140,KX410C,Other,33,Afghanistan,0MH71C,News,358125,172911,220


## <span style="color:#ff5f27">⚙️ Feature View Creation </span>


In [6]:
feature_view = fs.get_or_create_feature_view(
    name='retrieval',
    version=1,
    query=selected_features,
)

## <span style="color:#ff5f27">🏋️ Training Dataset </span>


In [7]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)
train_df.head(3)

Finished: Reading data from Hopsworks, using ArrowFlight (42.06s) 


,interaction_id,user_id,gender,age,country,video_id,category,views,likes,video_length
1,4380-37-7971,YU710V,Female,59,Gambia,5ZY75O,Comedy,7275,5936,171
2,3714-23-7897,IQ911P,Male,35,Sudan,6NE41R,Dance,66149,45789,238
4,7047-00-6140,KX410C,Other,33,Afghanistan,0MH71C,News,358125,172911,220


You will train your retrieval model with a subset of features.

For the query embedding you will use:
- `user_id`: ID of a user.
- `gender`: Gender of a user.
- `age`: age of a user.
- `country`: country if a user.

For the candidate embedding you will use:
- `video_id`: ID of a video.
- `category`: Video Category.
- `views`: Number of video views.
- `likes`: Number of video likes.
- `video_length`: Length of video.


In [8]:
def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col: df[col] for col in df})

BATCH_SIZE = 2048
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(val_df).batch(BATCH_SIZE).cache()

In [9]:
# Query Features 
user_id_list = train_df["user_id"].unique().tolist()
countries_list = train_df["country"].unique().tolist()
gender_list = train_df["gender"].unique().tolist()

# Item Features
video_id_list = train_df["video_id"].unique().tolist()
category_list = train_df["category"].unique().tolist()

print(f"⛳️ Number of users: {len(user_id_list)}")
print(f"⛳️ Number of items: {len(video_id_list)}")

⛳️ Number of users: 24983
⛳️ Number of items: 24990


## <span style="color:#ff5f27">🏰 Two Tower Model </span>


In [10]:
EMB_DIM = 16

In [11]:
class QueryTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.emb_dim = EMB_DIM
        self.user_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=user_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # You add an additional embedding to account for unknown tokens.
                len(user_id_list) + 1,
                self.emb_dim
            )
        ])

        self.normalized_age = Normalization(axis=None)
        
        # Converts strings into integer indices (scikit-learn LabelEncoder analog)
        self.gender_tokenizer = StringLookup(
            vocabulary=gender_list,
            mask_token=None,
        )
        
        self.country_tokenizer = StringLookup(
            vocabulary=countries_list, 
            mask_token=None,
        )

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(self.emb_dim, activation="relu"),
            tf.keras.layers.Dense(self.emb_dim)
        ])

    def call(self, inputs):
        gender_embedding = tf.one_hot(
            self.gender_tokenizer(inputs["gender"]),
            len(gender_list),
        )
        
        country_embedding = tf.one_hot(
            self.country_tokenizer(inputs["country"]),
            len(countries_list),
        )
        
        concatenated_inputs = tf.concat([
            self.user_embedding(inputs["user_id"]),
            tf.reshape(self.normalized_age(inputs["age"]), (-1,1)),
            gender_embedding,
            country_embedding,
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


query_model = QueryTower()

query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[QUERY_FEATURES]
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))

<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[-0.07137153,  0.25015256, -0.20719832,  0.10795874, -0.04775844,
        -0.17112827, -0.25529203,  0.14370468,  0.20744579, -0.05836441,
        -0.05365835,  0.07151634, -0.0790488 ,  0.01260526,  0.19363274,
        -0.04510418]], dtype=float32)>

In [12]:
class ItemTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.emb_dim = EMB_DIM
        self.video_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=video_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # You add an additional embedding to account for unknown tokens.
                len(video_id_list) + 1,
                self.emb_dim,
            )
        ])
        
        # Converts strings into integer indices (scikit-learn LabelEncoder analog)
        self.category_tokenizer = StringLookup(
            vocabulary=category_list, 
            mask_token=None,
        )
        
        self.normalized_views = Normalization(axis=None)
        self.normalized_likes = Normalization(axis=None)
        self.normalized_video_length = Normalization(axis=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(self.emb_dim, activation="relu"),
            tf.keras.layers.Dense(self.emb_dim)
        ])

    def call(self, inputs):
        category_embedding = tf.one_hot(
            self.category_tokenizer(inputs["category"]),
            len(category_list),
        )

        concatenated_inputs = tf.concat([
            self.video_embedding(inputs["video_id"]),
            category_embedding,
            tf.reshape(self.normalized_views(inputs["views"]), (-1,1)),
            tf.reshape(self.normalized_views(inputs["likes"]), (-1,1)),
            tf.reshape(self.normalized_views(inputs["video_length"]), (-1,1)),
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs

    
item_model = ItemTower()

item_df = train_df[CANDIDATE_FEATURES]
item_df.drop_duplicates(subset="video_id", inplace=True)
item_ds = df_to_ds(item_df)

item_model(next(iter(item_ds.batch(1))))

<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[-1506.157  , -1316.9346 ,   110.54877,  1466.5878 , -1651.4055 ,
         -585.2615 ,   382.82886,  1285.9456 , -2920.3616 , -2068.8064 ,
          592.77454,  -448.13715,  -584.4979 ,   910.5513 ,  -813.92737,
         1641.4902 ]], dtype=float32)>

In [13]:
class TwoTowerModel(tf.keras.Model):
    def __init__(self, query_model, item_model):
        super().__init__()
        self.query_model = query_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=item_ds.batch(BATCH_SIZE).map(self.item_model)
            )
        )

    def train_step(self, batch) -> tf.Tensor:
        # Set up a gradient tape to record gradients.
        with tf.GradientTape() as tape:

            # Loss computation.
            user_embeddings = self.query_model(batch)
            item_embeddings = self.item_model(batch)
            loss = self.task(
                user_embeddings, 
                item_embeddings,
                compute_metrics=False,
            )

            # Handle regularization losses as well.
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        metrics = {
            "loss": loss,
            "regularization_loss": regularization_loss,
            "total_loss": total_loss
        }

        return metrics

    def test_step(self, batch) -> tf.Tensor:
        # Loss computation.
        user_embeddings = self.query_model(batch)
        item_embeddings = self.item_model(batch)

        loss = self.task(
            user_embeddings, 
            item_embeddings,
            compute_metrics=False,
        )

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

### <span style="color:#ff5f27">🏃🏻‍♂️ Model Training </span>

You'll train our model using the AdamW optimizer, which applies weight regularization during training.

In [14]:
# Create a TwoTowerModel with the specified query_model and item_model
model = TwoTowerModel(query_model, item_model)

# Define an optimizer using AdamW with a learning rate of 0.01
optimizer = tf.keras.optimizers.AdamW(
    weight_decay=0.001, 
    learning_rate=0.01,
)

# Compile the model using the specified optimizer
model.compile(optimizer=optimizer)

In [15]:
model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=5,
)

Epoch 1/5
391/391 [==============================] - 23s 53ms/step - loss: 532928.6647 - regularization_loss: 0.0000e+00 - total_loss: 532928.6647 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 12652.2236 - val_regularization_loss: 0.0000e+00 - val_total_loss: 12652.2236
Epoch 2/5
391/391 [==============================] - 9s 24ms/step - loss: 15611.9873 - regularization_loss: 0.0000e+00 - total_loss: 15611.9873 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.

## <span style="color:#ff5f27">🗄️ Upload Model to Model Registry </span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

In [16]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [17]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function()
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {
            "user_id": instances["user_id"],
            "gender": instances["gender"],
            "age": instances["age"],
            "country": instances["country"],
            "query_emb": query_emb,
        }

# wrap query_model:   query_model -> query_model_module
query_model = QueryModelModule(model.query_model)

In [18]:
# Define the input specifications for the instances
instances_spec = {
    'user_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='user_id'),   # Specification for user IDs
    'gender': tf.TensorSpec(shape=(None,), dtype=tf.string, name='gender'),     # Specification for gender
    'country': tf.TensorSpec(shape=(None,), dtype=tf.string, name='country'),    # Specification for country
    'age': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='age'),              # Specification for age
}

# Get the concrete function for the query_model's compute_emb function using the specified input signatures
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

# Save the query_model along with the concrete function signatures
tf.saved_model.save(
    query_model,           # The model to save
    "query_model",         # Path to save the model
    signatures=signatures, # Concrete function signatures to include
)

2024-04-04 08:47:51,434 INFO: Function `compute_emb` contains input name(s) table_handle, 15911, resource with unsupported characters which will be renamed to query_tower_sequential_string_lookup_none_lookup_lookuptablefindv2_table_handle, query_tower_sequential_embedding_embedding_lookup_15911, query_tower_sequential_1_dense_1_biasadd_readvariableop_resource in the SavedModel.
2024-04-04 08:47:56,650 INFO: Assets written to: query_model/assets


In [19]:
tf.saved_model.save(
    model.item_model,    # The model to save
    "candidate_model",   # Path to save the model
)

2024-04-04 08:47:58,497 WARNING: Skipping full serialization of Keras layer <keras.src.layers.preprocessing.normalization.Normalization object at 0x7f523adb3190>, because it is not built.
2024-04-04 08:47:58,497 WARNING: Skipping full serialization of Keras layer <keras.src.layers.preprocessing.normalization.Normalization object at 0x7f523adb3af0>, because it is not built.
2024-04-04 08:48:02,972 INFO: Assets written to: candidate_model/assets


In [20]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Infer input schema from data.
query_model_input_schema = Schema(query_df)

# Manually specify output schema.
query_model_output_schema = Schema([{
    "name": "query_embedding",
    "type": "float32",
    "shape": [EMB_DIM],
}])

query_model_schema = ModelSchema(
    input_schema=query_model_input_schema,
    output_schema=query_model_output_schema,
)

query_model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'user_id', 'type': 'object'},
   {'name': 'gender', 'type': 'object'},
   {'name': 'age', 'type': 'int64'},
   {'name': 'country', 'type': 'object'}]},
 'output_schema': {'tensor_schema': [{'name': 'query_embedding',
    'shape': '[16]',
    'type': 'float32'}]}}

In [21]:
# Sample a query example from the query DataFrame
query_example = query_df.sample().to_dict("records")

# Create a tensorflow model for the query_model in the Model Registry 
mr_query_model = mr.tensorflow.create_model(
    name="query_model",                                           # Name of the model
    description="Model that generates query embeddings from user features",  # Description of the model
    input_example=query_example,                                  # Example input for the model
    model_schema=query_model_schema,                              # Schema of the model
)

# Save the query_model to the Model Registry
mr_query_model.save("query_model")                                # Path to save the model

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://snurran.hops.works/p/11383/models/query_model/1


Model(name: 'query_model', version: 1)

In [22]:
# Define the input schema for the candidate_model based on item_df
candidate_model_input_schema = Schema(item_df)

# Define the output schema for the candidate_model, specifying the shape and type of the output
candidate_model_output_schema = Schema([{
    "name": "candidate_embedding",   # Name of the output feature
    "type": "float32",               # Data type of the output feature
    "shape": [EMB_DIM],              # Shape of the output feature
}])

# Combine the input and output schemas to create the overall model schema for the candidate_model
candidate_model_schema = ModelSchema(
    input_schema=candidate_model_input_schema,    # Input schema for the model
    output_schema=candidate_model_output_schema,  # Output schema for the model
)

# Sample a candidate example from the item DataFrame
candidate_example = item_df.sample().to_dict("records")

# Create a tensorflow model for the candidate_model in the Model Registry
mr_candidate_model = mr.tensorflow.create_model(
    name="candidate_model",                                        # Name of the model
    description="Model that generates candidate embeddings from video features",  # Description of the model
    input_example=candidate_example,                              # Example input for the model
    model_schema=candidate_model_schema,                          # Schema of the model
)

# Save the candidate_model to the Model Registry
mr_candidate_model.save("candidate_model")                        # Path to save the model

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://snurran.hops.works/p/11383/models/candidate_model/1


Model(name: 'candidate_model', version: 1)

---